# Q-SITE Classiq Coding Challenge - Lab 3

### Thrid Challenge: Mastering Quantum Comparisons: The Swap Test Challenge with Classiq

---
### Objective:
Welcome to the Swap Test Challenge! Your task is to implement the Swap Test algorithm, a fundamental quantum procedure for comparing quantum states. Using Classiq’s quantum programming framework, you will construct a quantum circuit that measures the overlap between two quantum states, allowing you to determine their similarity with high precision. Your goal is to understand and apply the principles of the Swap Test to extract meaningful results from quantum computations.

---


### **Swap Test Challenge**

In this challenge, you will implement and explore the Swap Test algorithm using Classiq's quantum programming framework. The Swap Test is a quantum algorithm used to determine the overlap between two quantum states. This challenge will guide you through constructing a quantum circuit that uses the Swap Test to compare two random quantum states, measure their overlap, and validate the results.

### **Understanding the Swap Test**

The Swap Test is a quantum function that checks the overlap between two quantum states. The inputs of the function are two quantum registers of the same size, $|\psi_1\rangle$ and $|\psi_2\rangle$, and it returns as an output a single "test" qubit whose state encodes the overlap between the two inputs:

$
|q\rangle_{test} = \alpha |0\rangle + \sqrt{1 - \alpha^2} |1\rangle,
$

with

$
\alpha^2 = \frac{1}{2} \left( 1 + |\langle\psi_1|\psi_2\rangle|^2 \right).
$

Thus, the probability of measuring the test qubit in the state $|0\rangle$ is 1 if the states are identical (up to a global phase) and 0.5 if the states are orthogonal to each other.

The quantum model for the Swap Test starts with an H gate on the test qubit, followed by swapping between the two states controlled on the test qubit (a controlled-SWAP gate for each of the qubits in the two states), and a final H gate on the test qubit.


### **Step 1: Prepare Two Random Quantum States**

Your first task is to prepare two random quantum states with the same number of qubits. These states will be the inputs to the Swap Test.


In [6]:
import numpy as np

# Seed for reproducibility
np.random.seed(23)

# Number of qubits
NUM_QUBITS = 3

# Generate random amplitudes for two quantum states
amps1 = 1 - 2 * np.random.rand(2**NUM_QUBITS)  # Vector of 2^3 numbers in the range [-1,1]
amps2 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps1 = amps1 / np.linalg.norm(amps1)  # Normalize the vector
amps2 = amps2 / np.linalg.norm(amps2)


### **Step 2: Implement the Swap Test**

Next, you will implement the Swap Test algorithm. This involves preparing the quantum states, applying the Swap Test, and synthesizing the quantum circuit.


In [7]:
from classiq import *
from classiq.execution import ExecutionPreferences

@qfunc
def main(test: Output[QBit]):
    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_amplitudes(amps1.tolist(), 0.0, state1)
    prepare_amplitudes(amps2.tolist(), 0.0, state2)
    swap_test(state1, state2, test)
    
    # Task 1: Prepare amplitudes for the first quantum state
    # Hint: Use the prepare_amplitudes function with amps1
    # https://docs.classiq.io/latest/explore/tutorials/getting_started/part2_state_preparation/
    ########################################
    # Your code here to prepare state1 with amps1
    
    ########################################
    
    # Task 2: Prepare amplitudes for the second quantum state
    # Hint: Use the prepare_amplitudes function with amps2
    ########################################
    # Your code here to prepare state2 with amps2
    
    ########################################
    
    # Task 3: Apply the Swap Test to compare state1 and state2
    ########################################
    # Your code here to apply the swap_test function by
    # swap_test(state1: qbit[], state2: qbit[], output test: qbit)
    # https://github.com/Classiq/classiq-library/blob/main/functions/open_library_definitions/swap_test.qmod
    
    ########################################

### **Step 3: Synthesize and Visualize the Circuit**

After constructing the circuit, synthesize and visualize it to ensure the Swap Test has been implemented correctly.


In [8]:
# Task 4: Create the quantum model and synthesize the circuit

qmod = create_model(main)### Your Answer ###

### In classiq, we can use this function to set execution preferences:
qmod = set_execution_preferences(
    qmod, execution_preferences=ExecutionPreferences(num_shots=100_000)
)

qprog = synthesize(qmod)### Your Answer ###

show(qprog)

Opening: https://platform.classiq.io/circuit/aa452662-3cfd-4576-a34c-09cc32b6856f?version=0.49.0


### **Step 4: Execute the Circuit and Verify Results**
Finally, execute the circuit to measure the overlap between the two quantum states. Then, compare the result from the Swap Test with the exact overlap computed classically.

Using the expected probability of measuring the state $|0\rangle$ as defined above:

$
\alpha^2 = \frac{1}{2} \left( 1 + |\langle\psi_1|\psi_2\rangle|^2 \right),
$

we extract the overlap:

$
|\langle\psi_1|\psi_2\rangle|^2 = \sqrt{2P\left(q_{\text{test}} = |0\rangle\right) - 1}.
$

The exact overlap is computed with the dot product of the two state vectors. Note that for the sake of this demonstration, we execute this circuit 100,000 times to improve the precision of the probability estimate. This is usually not required in actual programs.


In [9]:
write_qmod(qmod, "qsite_challenge_3", decimal_precision=15)

res = execute(qprog).result()

# Comparing the measured overlap with the exact overlap
overlap_from_swap_test = np.sqrt(
    2 * res[0].value.counts["0"] / sum(res[0].value.counts.values()) - 1
)
exact_overlap = np.abs(amps1 @ amps2)

print("States overlap from Swap-Test result:", overlap_from_swap_test)
print("States overlap from classical calculation:", exact_overlap)

RTOL = 0.05
assert np.isclose(
    overlap_from_swap_test, exact_overlap, RTOL
), f"""
The quantum result is too far from the classical one, by a relative tolerance of {RTOL}.
"""

States overlap from Swap-Test result: 0.6970509307073623
States overlap from classical calculation: 0.70196335130861


### Now save the QASM file for Grading!

In [10]:
qasm = QuantumProgram.from_qprog(qprog).qasm

# Specify the file path where you want to save the QASM file
file_path = "qsite_challenge_3.qasm"

# Save the QASM string to a file
with open(file_path, 'w') as file:
    file.write(qasm)

print(f"QASM file saved at: {file_path}")

QASM file saved at: qsite_challenge_3.qasm


### **Challenge Summary**

In this challenge, you have constructed and executed a quantum circuit using the Swap Test to measure the overlap between two quantum states. By comparing the quantum result with the classical calculation, you have demonstrated the accuracy and efficiency of quantum algorithms in state comparison tasks.

This exercise showcases how quantum circuits can be used to solve complex problems in quantum computing, such as state similarity detection, with precision and scalability.

🎉  Now, it's time to run your circuit and see the power of the Swap Test in action!

## Additional information

#### Created by : Louis Chen

#### Advised by : Tali Cohn, Eden Schirman  